## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df=pd.read_csv('/datasets/data.csv')
len(df)

21525

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

Каждая строка таблицы содержит информацию о клиентах банка и их характеристики по определённым критериям. Необходимо определить как влияет наличие детей, семейное положение и уровень дохода на возврат кредита в срок. Для нахождения ответа ценны столбцы children, family_status, total_income. Данные из столбца debt позволят узнать выплачен ли кредит.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))

In [7]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [8]:
def conversion_children_count(df):
    if df['children'] == 20:
        return 2
    if df['children'] == -1:
        return 1
    else:
        return df['children']

df['children'] = df.apply(conversion_children_count, axis=1)

In [9]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [10]:
df['days_employed']=df['days_employed'].apply(abs)

In [11]:
df['days_employed'].value_counts()

1574.202821      1105
1547.382223       509
365213.306266     414
2689.368353       147
520.848083          2
                 ... 
339.153770          1
2849.351119         1
5619.328204         1
448.829898          1
582.538413          1
Name: days_employed, Length: 19353, dtype: int64

### Вывод

Пропущенные значения найдены в столбцах days_employed и total_income и заменены, что говорит о том, что 10% клиентов не делятся важной информацией, так как эти данные могут помешать получению кредита. Также проверив информацию, выяснилось, что в столбце children есть значения 20 и -1 (приймим их за опечатки), в стобце dob_years значения 0, которые заменим на среднее значение. Так как столбец со значениями возраста клиента никак не влияет на будущий результат, мы проигнорируем данные столбца.

### Замена типа данных

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [13]:
df['days_employed']=df['days_employed'].astype('int')
df['total_income']=df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В результате просмотра типов данных таблицы мы увидели, что необходимо перевести 2 столбцы к типу данных int, так как данные уже находятся в float. 

### Обработка дубликатов

In [14]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [15]:
df['education']=df['education'].str.lower()

In [16]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [17]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

### Вывод

Дубликаты были найдены только в столбце с описанием образования и, приведя к нормальному виду, мы видим, что 70% кредитов берут клиенты имеющие среднее образование, дубликаты могли возникнуть из-за человеческого фактора.

### Лемматизация

In [18]:
from pymystem3 import Mystem
m = Mystem()
df['lemmas']=df['purpose'].apply(m.lemmatize)

def purpose_simplification (text):
    if 'свадьба' in text:
        return 'свадьба'
    if 'автомобиль' in text:
        return 'авто'
    if 'образование' in text:
        return 'образование'
    if 'жилье' or 'недвижимость' in text:
        return 'ком. недвижимость'
    
df['lemmas']=df['lemmas'].apply(purpose_simplification)

### Вывод

Просмотрев начальные данные, мы можем выделить отдельные группы по целям получения кредита клиентами. Исходя из этого формируются 4 основные группы необходимые для удобства восприятия информации.

### Категоризация данных

In [3]:
def children_group(children):
    if children==0:
        return 'нет детей'
    if children==1:
        return '1 ребенок'
    if children==2:
        return '2 ребенка'
    if children >= 2:
        return 'многодетная семья'

df['children_group']=df['children'].apply(children_group)

### Вывод

Категоризация дает возможность разделить данные по опеделенным условиям в нашем варианте по количеству детей.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
df.groupby('children_group')['debt'].agg(['count','mean'])

,count,mean
children_group,,
1 ребенок,4865,0.091470
2 ребенка,2131,0.094791
многодетная семья,380,0.081579
нет детей,14149,0.075129


In [4]:
df.pivot_table(index=['children_group'], values='debt',aggfunc=['count','mean'])

,count,mean
,debt,debt
children_group,,
1 ребенок,4818,0.092154
2 ребенка,2055,0.094404
многодетная семья,456,0.085526
нет детей,14149,0.075129


### Вывод

При отсутствии детей процент погашения кредитов больше, чем при наличии детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
df.groupby('family_status')['debt'].agg(['count','mean'])

,count,mean
family_status,,
Не женат / не замужем,2813,0.097405
в разводе,1195,0.071130
вдовец / вдова,960,0.065625
гражданский брак,4177,0.092890
женат / замужем,12380,0.075202


### Вывод

Следовательно, процент задолженности кредита больше у холостых людей.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
df['total_income_group'] = pd.cut(df['total_income'],2)
df.groupby('total_income_group')['debt'].agg(['count','mean'])

,count,mean
total_income_group,,
"(18422.063, 1143135.5]",21511,0.080889
"(1143135.5, 2265604.0]",14,0.071429


### Вывод

Исходя из таблицы можно сделать вывод, что, хоть и клиентов с большим ежемесячным доходом намного меньше, процент задолженности у клиентов с средним доходом не намного больше.

- Как разные цели кредита влияют на его возврат в срок?

In [24]:
df.groupby('lemmas')['debt'].agg(['count','mean'])

,count,mean
lemmas,,
авто,4315,0.093395
ком. недвижимость,10840,0.072140
образование,4022,0.091994
свадьба,2348,0.079216


### Вывод

В итоге, мы видим, что клиенты, указывающие своей причиной недвижимость, быстрее выплачивают кредит.

### Шаг 4. Общий вывод

Просмотрев начальную таблицу видно множество недочетов, которые мы исправили: заменили пропуски в столбцах days_employed и total_income на медианное значение, заменили тип данных float на int, привели столбец education к нижнему регистру, разбили столбец purpose на группы с помощью лемматизации, разделили данные столбца children по кол-ву детей с помощью категоризации и т.д. Затем проверили зависимость возврата кредита в срок от 1) кол-ва детей, 2) семейного положения, 3) уровня дохода, 4) цели получения кредита. 
    
Рекомендации: клиенты, не имеющие детей, находящиеся в браке и нуждающиеся в помощи с недвижимостью, возвращают кредит быстрее остальных. 